In [ ]:
import torchvision.transforms as transforms
import pyrealsense2.pyrealsense2 as rs
from threading import Thread
import numpy as np
import ipywidgets
import time
import cv2

clockIsVisible = 0
color = 0

In [ ]:
def FindMyCameras():
    ctx = rs.context()
    devices = [x for x in list(ctx.query_devices())]
    idxs = [str(x).find('S/N') for x in devices]
    devices = [str(x)[y+5:y+17] for x,y in zip(devices,idxs)]
    return devices

In [ ]:
def PrepareCamera(pipeline, serialNo, useRGB):
    config = rs.config()
    pipeline_wrapper = rs.pipeline_wrapper(pipeline)
    pipeline_profile = config.resolve(pipeline_wrapper)
    config.enable_device(serialNo)
    config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
    
    if useRGB:
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
    return config

In [ ]:
def InitializeCameras(number_of_cameras):
    cameras = FindMyCameras()
    frontCam = rs.pipeline()
    frontCamConfig = PrepareCamera(frontCam, cameras[0], useRGB=True)
    profile_front = frontCam.start(frontCamConfig)
    depth_sensor = profile_front.get_device().first_depth_sensor()
    front_depth_scale = depth_sensor.get_depth_scale()
    
    if number_of_cameras == 2:
        backCam = rs.pipeline()
        backCamConfig = PrepareCamera(backCam, cameras[1], useRGB=False)
        profile_back = backCam.start(backCamConfig)
        depth_sensor = profile_back.get_device().first_depth_sensor()
        back_depth_scale = depth_sensor.get_depth_scale()
        print('Cameras ready to start streaming! :)')
        return frontCam, backCam, front_depth_scale, back_depth_scale
    
    print('Camera ready to start streaming! :)')
    return frontCam, frontCam, front_depth_scale, front_depth_scale

In [ ]:
''' Reads One Camera and returns Frames as arrays'''
def GetFrames(camera, getColor=False):
    align_to = rs.stream.color
    align = rs.align(align_to)
    frames = camera.wait_for_frames()
    aligned_frames = align.process(frames)
    depth_frame = aligned_frames.get_depth_frame()
    depth_image = np.asanyarray(depth_frame.get_data())

    if getColor:
        color_frame = aligned_frames.get_color_frame()
        color_image = np.asanyarray(color_frame.get_data())
        color_image = cv2.flip(color_image, 1)
        return color_image, depth_image
    else:
        return depth_image

In [ ]:
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

## Prepare Widget and Initialize Cameras

In [ ]:
frontalCamera, backwardCamera, frontScale, backScale = InitializeCameras(1)

## Load ResNet Model

In [ ]:
# Download Model from torchvision
from torchvision.models import detection
from torch2trt import torch2trt
import torch
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = detection.fasterrcnn_resnet50_fpn(pretrained=True).to(DEVICE)
model.eval()